In [1]:
# Michael A. Alcorn (malcorn@redhat.com)
# A (slightly modified) implementation of RankNet as described in [1].
#   [1] http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf
#   [2] https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf

import numpy as np

from tensorflow.keras import backend

from tensorflow.keras.layers import Activation, Dense, Input, Subtract
from tensorflow.keras.models import Model


In [11]:
INPUT_DIM = 10

# Model.
h_1 = Dense(128, activation = "relu")
h_2 = Dense(64, activation = "relu")
h_3 = Dense(32, activation = "relu")
s = Dense(1)

# Relevant document score.
rel_doc = Input(shape = (INPUT_DIM, ), dtype = "float32")
h_1_rel = h_1(rel_doc)
h_2_rel = h_2(h_1_rel)
h_3_rel = h_3(h_2_rel)
rel_score = s(h_3_rel)

# Irrelevant document score.
irr_doc = Input(shape = (INPUT_DIM, ), dtype = "float32")
h_1_irr = h_1(irr_doc)
h_2_irr = h_2(h_1_irr)
h_3_irr = h_3(h_2_irr)
irr_score = s(h_3_irr)

# Subtract scores.
diff = Subtract()([rel_score, irr_score])

# Pass difference through sigmoid function.
prob = Activation("sigmoid")(diff)

# Build model.
model = Model(inputs = [rel_doc, irr_doc], outputs = prob)
model.compile(optimizer = "adadelta", loss = "binary_crossentropy")


In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          256         input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           8256        dense[0][0]                      
          

In [12]:
# Fake data.
N = 100
X_1 = 2 * np.random.uniform(size = (N, INPUT_DIM))
X_2 = np.random.uniform(size = (N, INPUT_DIM))
y = np.ones((X_1.shape[0], 1))


In [13]:
# Train model.
NUM_EPOCHS = 10
BATCH_SIZE = 10
history = model.fit([X_1, X_2], y, batch_size = BATCH_SIZE, epochs = NUM_EPOCHS, verbose = 1)

# Generate scores from document/query features.
get_score = backend.function([rel_doc], [rel_score])
#print get_score([X_1])
#get_score([X_2])

Epoch 1/10
100/100 [==============================] - 1s 8ms/step - loss: 0.5093
Epoch 2/10
100/100 [==============================] - 0s 353us/step - loss: 0.2360
Epoch 3/10
100/100 [==============================] - 0s 430us/step - loss: 0.1047
Epoch 4/10
100/100 [==============================] - 0s 427us/step - loss: 0.0565
Epoch 5/10
100/100 [==============================] - 0s 471us/step - loss: 0.0381
Epoch 6/10
100/100 [==============================] - 0s 418us/step - loss: 0.0284
Epoch 7/10
100/100 [==============================] - 0s 887us/step - loss: 0.0221
Epoch 8/10
100/100 [==============================] - 0s 392us/step - loss: 0.0176
Epoch 9/10
100/100 [==============================] - 0s 306us/step - loss: 0.0143
Epoch 10/10
100/100 [==============================] - 0s 382us/step - loss: 0.0117


In [88]:
rel_doc.shape

TensorShape([Dimension(None), Dimension(100)])

In [15]:
#print np.mean(get_score([X_test]))
#print np.mean(get_score([X_test_2]))


#print get_score([X_test[0].reshape(1,-1)])[0]
get_score([X_1])

[array([[13.151611 ],
        [12.755197 ],
        [17.037495 ],
        [12.849141 ],
        [16.149529 ],
        [12.821463 ],
        [19.512638 ],
        [15.077787 ],
        [18.58548  ],
        [13.793613 ],
        [14.040407 ],
        [14.446678 ],
        [15.769969 ],
        [11.526705 ],
        [17.27346  ],
        [17.774662 ],
        [17.023296 ],
        [15.51837  ],
        [12.174841 ],
        [13.936768 ],
        [15.150005 ],
        [16.539719 ],
        [16.822834 ],
        [11.444159 ],
        [ 9.691441 ],
        [14.895153 ],
        [16.674446 ],
        [16.584354 ],
        [16.927    ],
        [14.86408  ],
        [16.998953 ],
        [13.20396  ],
        [16.941227 ],
        [22.66306  ],
        [17.235527 ],
        [15.036436 ],
        [14.780407 ],
        [17.75266  ],
        [11.220555 ],
        [14.835354 ],
        [15.517542 ],
        [18.49413  ],
        [16.1913   ],
        [13.993706 ],
        [15.943951 ],
        [1

In [7]:
from collections import Counter
X_test = 1.2 * np.random.uniform(size = (50, INPUT_DIM))
X_test_2 =  np.random.uniform(size = (50, INPUT_DIM))

#Counter((model.predict([X_test, X_test_2]) >= 0.5).flatten())

model.predict([X_test[0].reshape(1,-1), X_test_2[0].reshape(1,-1)])


array([[0.81831026]], dtype=float32)